In [1]:
import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils
plt.rcParams['figure.figsize'] = (13, 5)

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from collections import Counter
from sklearn.preprocessing import RobustScaler,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from scipy import interp
from itertools import cycle
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall
from scikitplot.metrics import plot_cumulative_gain
from scikitplot.metrics import plot_lift_curve
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
tracks = utils.load('data/fma_metadata/tracks.csv')

tracks.shape

(106574, 52)

In [3]:
df=tracks[[('track', 'comments'),('track', 'favorites'),('track', 'interest'),('track', 'listens'),('set', 'subset')]]

In [4]:
df=df.loc[df[('set', 'subset')]!='large']

In [5]:
df=df.drop([('set', 'subset')], axis = 1)


In [6]:
df

track                           
         comments favorites interest listens
track_id                                    
2               0         2     4656    1293
3               0         1     1470     514
5               0         6     1933    1151
10              0       178    54881   50135
134             0         3     1126     943
...           ...       ...      ...     ...
155297          0         3     1463    1049
155298          0         0      706     590
155306          0         1      497     435
155307          0         0      630     571
155314          0         0      778     652

[25000 rows x 4 columns]

In [7]:
attributes = [col for col in df.columns if (col != ('track', 'favorites'))] 



X = df[attributes].values
y = df[('track', 'favorites')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X

array([[   0, 4656, 1293],
       [   0, 1470,  514],
       [   0, 1933, 1151],
       ...,
       [   0,  497,  435],
       [   0,  630,  571],
       [   0,  778,  652]], dtype=int64)

In [9]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [19]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [20]:
y_pred_train = reg.predict(X_train)

print('R2: %.3f' % r2_score(y_train, y_pred_train))
print('MSE: %.3f' % mean_squared_error(y_train, y_pred_train))
print('MAE: %.3f' % mean_absolute_error(y_train, y_pred_train))

R2: 0.835
MSE: 45.882
MAE: 2.459


In [21]:
y_pred = reg.predict(X_test)

print('R2: %.3f' % r2_score(y_test, y_pred))
print('MSE: %.3f' % mean_squared_error(y_test, y_pred))
print('MAE: %.3f' % mean_absolute_error(y_test, y_pred))

R2: 0.692
MSE: 26.688
MAE: 2.314


In [22]:
print('Coefficients: \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

Coefficients: 
 [5.80387164e+00 2.58210201e-04 6.51013736e-04]
Intercept: 
 0.8574019960255566


## RIDGE

In [17]:
parameters = {'alpha':[0.0001,0.001,0.01,0.1,1,10],}

# define the model/ estimator
model = Ridge(random_state=0,normalize=True)

# define the grid search
clf1= GridSearchCV(model, parameters, scoring='neg_mean_squared_error',cv=5)

clf1 = clf1.fit(X_train, y_train)
report(clf1.cv_results_, n_top=10)


Model with rank: 1
Mean validation score: -59.775 (std: 33.307)
Parameters: {'alpha': 0.0001}

Model with rank: 2
Mean validation score: -59.785 (std: 33.319)
Parameters: {'alpha': 0.001}

Model with rank: 3
Mean validation score: -59.886 (std: 33.445)
Parameters: {'alpha': 0.01}

Model with rank: 4
Mean validation score: -61.024 (std: 35.322)
Parameters: {'alpha': 0.1}

Model with rank: 5
Mean validation score: -84.464 (std: 76.202)
Parameters: {'alpha': 1}

Model with rank: 6
Mean validation score: -208.215 (std: 220.777)
Parameters: {'alpha': 10}



In [27]:
reg = Ridge(alpha=0.001,normalize=True,random_state=0)
reg.fit(X_train, y_train)

Ridge(alpha=0.001, random_state=0)

In [28]:
y_pred_train = reg.predict(X_train)

print('R2: %.3f' % r2_score(y_train, y_pred_train))
print('MSE: %.3f' % mean_squared_error(y_train, y_pred_train))
print('MAE: %.3f' % mean_absolute_error(y_train, y_pred_train))

R2: 0.835
MSE: 45.882
MAE: 2.459


In [29]:
y_pred = reg.predict(X_test)

print('R2: %.3f' % r2_score(y_test, y_pred))
print('MSE: %.3f' % mean_squared_error(y_test, y_pred))
print('MAE: %.3f' % mean_absolute_error(y_test, y_pred))

R2: 0.692
MSE: 26.688
MAE: 2.314


In [30]:
print('Coefficients: \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

Coefficients: 
 [5.80386942e+00 2.58210221e-04 6.51013741e-04]
Intercept: 
 0.8574019906823231


## LASSO

In [36]:
parameters = {'alpha':[0.00001,0.0001,0.001,0.01,0.1,1,10],}

# define the model/ estimator
model = Lasso(random_state=0,normalize=True)

# define the grid search
clf1= GridSearchCV(model, parameters, scoring='neg_mean_squared_error',cv=5)

clf1 = clf1.fit(X_train, y_train)
report(clf1.cv_results_, n_top=10)


Model with rank: 1
Mean validation score: -59.778 (std: 33.314)
Parameters: {'alpha': 1e-05}

Model with rank: 2
Mean validation score: -59.819 (std: 33.402)
Parameters: {'alpha': 0.0001}

Model with rank: 3
Mean validation score: -60.291 (std: 34.353)
Parameters: {'alpha': 0.001}

Model with rank: 4
Mean validation score: -67.818 (std: 50.660)
Parameters: {'alpha': 0.01}

Model with rank: 5
Mean validation score: -243.964 (std: 299.523)
Parameters: {'alpha': 0.1}

Model with rank: 6
Mean validation score: -278.001 (std: 285.982)
Parameters: {'alpha': 1}

Model with rank: 6
Mean validation score: -278.001 (std: 285.982)
Parameters: {'alpha': 10}



In [26]:
reg = Lasso(alpha=0.00001,normalize=True,random_state=0)
reg.fit(X_train, y_train)

Lasso(alpha=1e-05, normalize=True, random_state=0)

In [28]:
y_pred_train = reg.predict(X_train)

print('R2: %.3f' % r2_score(y_train, y_pred_train))
print('MSE: %.3f' % mean_squared_error(y_train, y_pred_train))
print('MAE: %.3f' % mean_absolute_error(y_train, y_pred_train))

R2: 0.835
MSE: 45.882
MAE: 2.459


In [29]:
y_pred = reg.predict(X_test)

print('R2: %.3f' % r2_score(y_test, y_pred))
print('MSE: %.3f' % mean_squared_error(y_test, y_pred))
print('MAE: %.3f' % mean_absolute_error(y_test, y_pred))

R2: 0.692
MSE: 26.690
MAE: 2.314


In [30]:
print('Coefficients: \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

Coefficients: 
 [5.80205337e+00 2.58209354e-04 6.50918752e-04]
Intercept: 
 0.8577227917996244
